In [36]:
from pathlib import Path
from python.complex import complex_t
from python.util import *
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants
import time

In [37]:
# Инициализация динамической библиотеки
lib_name = "build/libAntennaArray.so"
c_lib = InitializeLibrary(lib_name)

In [38]:
# Инициализация переменных
N = 16
freq_0 = 3 * 10**9
wave_length = (constants.c) / freq_0
wave_num = 2 * np.pi / wave_length

In [39]:
# Расчет необходимых данных для расчета
theta_range = np.linspace(0, 2*np.pi, 1001)
f_arr = [complex_t(1, 0) for _ in theta_range]
delta_x = CalculateDeltaX(wave_length, np.pi / 6)
L = delta_x * (N - 1)
x_arr = np.array([(i * delta_x) - L/2 for i in range(N)])

In [40]:
# Конвертация питоновских списков/массивов в C-массив
c_theta_range = ListToCDoubleArray(theta_range)
c_f_arr = ListToCComplexArray(f_arr)
c_x_arr = ListToCDoubleArray(x_arr)

In [41]:
# Расчет диаграммы направленности c динамической библиотекой

CYCLES = 1000

abs_rad_pattern = np.zeros_like(theta_range)

c_total_time = 0
# разогревочный прогон
for _ in range(CYCLES // 10):
    radiation_pattern = c_lib.Calculate1DAntennaArray(ct.c_int(N), ct.c_int(
        theta_range.size), c_f_arr, c_x_arr, c_theta_range, ct.c_double(wave_num))
    c_lib.FreeComplexArr(radiation_pattern)

for _ in range(CYCLES):
    start = time.time()
    radiation_pattern = c_lib.Calculate1DAntennaArray(ct.c_int(N), ct.c_int(
        theta_range.size), c_f_arr, c_x_arr, c_theta_range, ct.c_double(wave_num))
    end = time.time()
    c_total_time += end - start
    c_lib.FreeComplexArr(radiation_pattern)
print(c_total_time / CYCLES)

0.002892033815383911


In [42]:
# Расчет диаграммы направленности на питоне

python_total_time = 0
# разогревочный прогон
for _ in range(CYCLES // 10):
    radiation_pattern = Calculate1DAntennaArray(N, f_arr, x_arr, theta_range, wave_num)
        
for _ in range(CYCLES):
    start = time.time()
    radiation_pattern = Calculate1DAntennaArray(N, f_arr, x_arr, theta_range, wave_num)
    end = time.time()
    python_total_time += end - start

print(python_total_time / CYCLES)

0.06612304091453552


In [45]:
print(f"Динамическая библиотека работат в примерно {round(python_total_time / c_total_time)} раз быстрее")

Динамическая библиотека работат в примерно 23 раз быстрее


рассчитать КНД